<center>
<a href="http://www.udem.edu.co/"><img src="https://www.universidadesvirtuales.com.co/logos/original/logo-universidad-de-medellin.png"></a>
<h1>Reconocimiento de Patrones I</h1>
<h3>2018-2</h3>
<h2>Proyecto de Aula: Caracterización de moras</h2>
</center>
---

## Estudiantes
---

Nombre: Ana María Sosa 

Identificación: 1017235052
    
Nombre: Melisa Morales Gómez
    
Identificación: 1035875351

Nombre: Miguel Angel Mejia
    
Identificación: 1036646927


## Descripción del proyecto
---

A través de tres clases de moras, las cuales se encuentran entre malas, buenas y regulares, se realizará a partir de las imágenes de estas moras una medición a través de regionprops lo que nos generará unas características entre las cuales se utilizarán el area, area convexa, excentricidad, etc. A partir de estos datos se se podrá resolver el problema, el cual será clasificar las moras en sus tres calidades.

Este dataset consta de 240 imágenes, 80 de cada calidad. Fue tomado de un proyecto en el cuál se elaboró un sistema clasificador automático de moras dentro de esas tres clases (buenas, regulares y malas), desarrollado en el marco curso de procesamiento digital de imágenes de la Universidad de Antioquia.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from PIL import Image

from skimage import data
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage import io
from skimage import exposure
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.segmentation import clear_border

from sklearn import cross_validation
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

/Users/anasosa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Método Otsu

In [2]:
def otsu(mora):
    p2, p98 = np.percentile(mora, (1, 98))
    mora = exposure.rescale_intensity(mora, in_range=(p2, p98)) 
    
    thresh1 = threshold_otsu(mora)
    moraBinarizada = mora > thresh1

    return moraBinarizada

## Lista de Moras Binarizadas y Etiquetadas

In [3]:
#Se declara la lista para las imágenes y las etiquetas posibles
moras = []

def crear_moras(clase):
    # Se recorre por la cantidad de imagenes que hay de cada clase que son 80
    for j in range(1,81):
        imagen = clase + "/ImagenMora" + str(j) +".jpg"
        mora = Image.open(imagen)
        mora = mora.convert('L')  
        mora = np.asarray(mora,dtype=np.float32)
         
        tupla_mora = (otsu(mora), clase)
        
        moras.append(tupla_mora) 
        
crear_moras('Moras buenas')
crear_moras('Moras malas')
crear_moras('Moras regulares')

In [4]:
print(len(moras))

6


## Caracterización con RegionProps

In [ ]:
X = [] 
y = []

for i in range(len(moras)):
    print('Caracterizando mora nuúmero', i+1, 'de la clase:', moras[i][1])
    etiqueta = moras[i][1]
    label_img = label(moras[i][0])
    regions = regionprops(label_img)
    
    caracteristicas = [] 
    area = 0
    numero_region = 0
    cont = 0
    for j in regions:
        if j.area > area:
            area = j.area                
            numero_region = cont  
            cont += 1

    caracteristicas.append(area)      
    caracteristicas.append(regions[numero_region].bbox_area)       
    caracteristicas.append(regions[numero_region].convex_area)      
    caracteristicas.append(regions[numero_region].eccentricity)        
    caracteristicas.append(regions[numero_region].equivalent_diameter)     
    caracteristicas.append(regions[numero_region].euler_number)
    caracteristicas.append(regions[numero_region].extent)
    caracteristicas.append(regions[numero_region].filled_area)   
    caracteristicas.append(regions[numero_region].orientation)  
    caracteristicas.append(regions[numero_region].perimeter)  
    caracteristicas.append(regions[numero_region].solidity) 

    X.append(caracteristicas)

    if etiqueta == 'Moras buenas':         
        y.append(2)    
    elif etiqueta == 'Moras regulares':  
        y.append(1)
    else:
        y.append(0)
    

Caracterizando mora # 1 de la clase:  Moras buenas
Caracterizando mora # 2 de la clase:  Moras buenas
Caracterizando mora # 3 de la clase:  Moras malas
Caracterizando mora # 4 de la clase:  Moras malas
Caracterizando mora # 5 de la clase:  Moras regulares


In [ ]:
# Se crea la instancia de la clase para realizar cross validation usando Kfolds
kf = cross_validation.KFold(len(moras), n_folds=10)

# Se instancia la variable para almacenar los datos
data = []

# Se prueban diferentes valores de C y Gamma para maquinas de vectors de soporte(SVC)
for c in (1,10,100):
    row = [c]
    rowError = [c]
    for Gamma in (0.0001, 0.001, 0.01, 0.1):
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)
        clf = SVC(kernel='linear', C=c, gamma = Gamma, shrinking = True)    
        clf.fit(X_train, y_train)           
        scoresSVC = cross_validation.cross_val_score(clf, X_test, y_test, cv=10)
        rowEf = str("%0.2f (+/- %0.2f)" % (scoresSVC.mean(), scoresSVC.std() * 2))
        rowErr = str("%0.2f (+/- %0.2f)" % (1 - scoresSVC.mean(), scoresSVC.std() * 2))
        row.append(rowEf)
        row.append(rowErr)
    data.append(row)

In [ ]:
# Se realizan 10 iteraciones usando cross validation para el classificador de arboles de decición
suma = 0
for k, (train, test) in enumerate(kf):
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)
    clf = tree.DecisionTreeClassifier(max_leaf_nodes = 100)
    clf = clf.fit(X_train, y_train)
    suma += clf.score(X_test, y_test)
scoresTree = cross_validation.cross_val_score(clf, X_test, y_test, cv=10)

In [ ]:
# Se corre  el clasificador "Random forest" con 10, 20, 30, 40 y 100 arboles con cross validation de 10 Kfolds
forestData = []
for N in (10, 20, 30, 40, 100):
    forestRow = []
    suma = 0
    for k, (train, test) in enumerate(kf):
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)
        clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)  
        clf.fit(X_train, y_train)
        suma += clf.score(X_test, y_test)
    scores = cross_validation.cross_val_score(clf, X_test, y_test, cv=10)    
    forestData.append([N, ("%0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)),("%0.2f (+/- %0.2f)" % (1 - scores.mean(), scores.std() * 2))])

In [ ]:
print("Maquinas de vectores de soporte (SVC) Errores")
pd.DataFrame(data, columns=['C', '0.0001', 'error', '0.001', 'error', '0.01', 'error', '0.1', 'error' ])

In [ ]:
print("Arbol de decición")
print(("Eficiencia: %0.2f (+/- %0.2f)" % (scoresTree.mean(), scoresTree.std() * 2)),(" - Error: %0.2f (+/- %0.2f)" % (1 - scoresTree.mean(), scoresTree.std() * 2)))

In [ ]:
print("Random forest")
pd.DataFrame(forestData, columns=['# Arboles', 'Eficiencia', 'Error'])

## Conclusion

Segun los resultados obtenidos de los tre modelos, el que presenta mejor eficiencia es "Random forest" con una configuración de 20 a 30 arboles, con un 70% de eficiencia en promedio y un intervalo de confianza relativamente bajo en comparación con los demas modelos (entre .28 y .32). realizando varias pruebas, los resultados apuntan a esta configuración, aunque en ocaciones con menos o más arboles puede tender a mejorar la eficiencia.